# Dimensionality Reduction + KNN
In the last tutorial I alluded to whether or not we needed to include features that have roughly the same spread of data. That is, if 90% of values in a given feature are the same, is that feature actually important? What we did in the last tutorial with data preprocessing involved a fair bit of manual feature reduction. In this tutorial you will be looking at a couple of ways to automatically reduce the number of features. You will also be learning about a type of classification based on neighboring data.

## MNIST

MNIST.... again? Ya, I know you've already seen MNIST in the multiclass classification tutorial, but I think it will be a good one to introduce dimensionality reduction and KNN. In this tutorial you will learn:

- KNN
- PCA
- t-SNE

Let's get started!

### KNN

K nearest neighbors (KNN) is a pretty simple algorithm. It's actually your first introduction to unsupervised learning as well. Unsupervised learning techniques don't rely on a label. You might be asking, why use it then? After all, we have the labels, so why not use them? KNN is only searching for the neraest points, but we can use it in conjunction with the label to help determine the class label. Let's start with a simple example by creating a 2D space with 9 points and 3 classes.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

points = [
    (-2, 2), (-3, 2), (-3, 3), (-2, 3), # Class 0
    (-2, -1), (-3, -1), (-3,-2),        # Class 1
    (1, 2), (2, 2), (2, 3)              # Class 2
]

classes = [0, 0, 0, 0, 1, 1, 1, 2, 2, 2]

x, y = zip(*points)
plt.figure(figsize=(10,10))
sns.scatterplot(x=x, y=y, c=classes, s=500)


Now let's assume a new point comes in and we don't know what class it belongs to. We can make an educated guess that it belongs whatever class it's near. In fact we can make it a bit more robust by taking into account the `k` nearest points. This is KNN. Let's see what this looks like by adding a new point and using KNN to classify which class it should belong to.

In [ ]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

new_point = (0, 0)
x = np.array(points)

f, axs = plt.subplots(4, 2, figsize=(20,40))
ks = [1, 2, 3, 4, 5, 6, 7, 8]

def plot_lines_from_point_to_neighbors(point, neighboring_points, ax, dists):
    # Flatten and then grab the x, y coords
    neighbors = neighboring_points.ravel()
    xn = x[neighbors][:, 0].tolist()
    yn = x[neighbors][:, 1].tolist()

    # Plot a line segment from from the new point to the neighbor
    for p in zip(xn, yn):
        src = point
        dst = p
        ax.plot([src[0], dst[0]], [src[1], dst[1]])
    
    # Plot distances of all points
    for dist, dst in zip(dists, x.tolist()):
        ax.text(dst[0]-0.015, dst[1]+0.25, f'{dist:.3f}')


for i, k in enumerate(ks):
    # Store the training data
    knn = NearestNeighbors(n_neighbors=k, metric='euclidean', algorithm='brute')
    knn.fit(x)

    # Find the nearest points
    data_point = np.array(new_point).reshape(-1, 2)
    dists, neighbors = knn.kneighbors(data_point, return_distance=True)
   
    # Grab the most frequent class
    neighboring_classes = np.array(classes)[neighbors].ravel()
    new_class = np.bincount(neighboring_classes).argmax()

    xs = x[:, 0].ravel().tolist() + [new_point[0]]
    ys = x[:, 1].ravel().tolist() + [new_point[1]]
    row, col = i // 2, i % 2
    axs[row, col].scatter(xs, ys, c=classes + [new_class], s=350)
    axs[row, col].set_title(f'k={k} predicts {new_class}')

    # Compute the distance of the points purely for plotting reasons
    dists = np.linalg.norm(x - np.array(new_point), axis=1).tolist()
    plot_lines_from_point_to_neighbors(new_point, neighbors, axs[row, col], dists)

Take a look at the above plot. Does it make sense? For the most part it does, but take a look at `k=1` and `k=2`. Notice that in both it predicts class 1 instead of class 2 even though they're the same distance. This is because `numpy` needs a way to break ties when we look for the frequency. In this case it ends up selecting the one with the smaller numerical class. That's probably fine in this case. Now take a look at `k=3`. In this case the 3 nearest neighbors are a neighbor from each class: `0, 1, 2`. It might be a bit hard to tell but class 0 is actually the furthest away, however, because of how `numpy` is breaking ties, it doesn't take that into account. This is also probably fine in the long run. Yes, you might end up with an incorrect classification here or there, but overall this probably won't matter. If it does, you can make a smarter selection by trying to break ties based on the average distance of those neighbors. Here's what that would look like.

In [ ]:
from collections import Counter
import pandas as pd

def get_class(dists, classes):
    df = pd.DataFrame(np.column_stack((dists.T, classes.T)), columns=['dists', 'classes'])
    # Create a mapping from class -> avg dist
    avg_dists = {curr_class: df.loc[df['classes'] == curr_class, 'dists'].mean() for curr_class in df['classes'].unique().tolist()}
    classes = classes.tolist()
    count = Counter(classes)
    # Sort by smallest negative count, i.e. largest count with tie breaker going
    # to the smallest avg distv and then finally just choosing smallest class
    sort = sorted(classes, key = lambda c: (-count[c], avg_dists[c], c))
    return sort[0]


new_point = (0, 0)
x = np.array(points)

f, axs = plt.subplots(4, 2, figsize=(20,40))
ks = [1, 2, 3, 4, 5, 6, 7, 8]

for i, k in enumerate(ks):
    # Store the training data
    knn = NearestNeighbors(n_neighbors=k, metric='euclidean', algorithm='brute')
    knn.fit(x)

    # Find the nearest points
    data_point = np.array(new_point).reshape(-1, 2)
    dists, neighbors = knn.kneighbors(data_point, return_distance=True)
   
    # Grab the most frequent class
    neighboring_classes = np.array(classes)[neighbors].ravel()
    new_class = get_class(dists, neighboring_classes)

    xs = x[:, 0].ravel().tolist() + [new_point[0]]
    ys = x[:, 1].ravel().tolist() + [new_point[1]]
    row, col = i // 2, i % 2
    axs[row, col].scatter(xs, ys, c=classes + [new_class], s=350)
    axs[row, col].set_title(f'k={k} predicts {new_class}')

    # Compute the distance of the points purely for plotting reasons
    dists = np.linalg.norm(x - np.array(new_point), axis=1).tolist()
    plot_lines_from_point_to_neighbors(new_point, neighbors, axs[row, col], dists)

As you can see, KNN relies on `k` and how you decide to select the majority class. You may also have noticed that when there are a number of points at the same distance, sklearn will choose one over the other. This is fine since in the long wrong it's unlikely to negatively impact accuracy.

Now that you've seen how KNN works, let's apply it on `MNIST`. Rather than having the 2 features (x and y location) that you just saw above, you're now going to have 784 features (28 x 28 pixels). Because of this there are a lot more computations that need to be done in order to determine the nearest neighbors. After all, you're not going to know the nearest ones unless you calculate the distance to all points. Let's see what this looks like.

In [ ]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
import numpy as np
mnist = fetch_openml('mnist_784', version=1, cache=True)

X = mnist.data.to_numpy()
y = mnist.target.to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=10000, random_state=0xC0FFEE)

print(X_train.shape)
print(X_test.shape)

Start by looking at the type of data we receive.

In [ ]:
print('X_train type', X_train.dtype)
print('y_train type', y_train.dtype)
print('min/max', X_train.min(), X_train.max())

In this case `X_train` is stored as a float with pixel values `[0, 255]` and `y_train` looks to be a string. Let's convert y to an int.

In [ ]:
y_train = y_train.astype(np.int32)
y_test  = y_test.astype(np.int32)

In [ ]:
def get_accuracy(knn, n, key_metric, X_test):
    # Calculate the neighbors for X_test
    dists, neighbors = knn.kneighbors(X_test, n_neighbors=n, return_distance=True)
    if key_metric == 'maximum_class':
        y_pred = [np.bincount(row).argmax() for row in y_train[neighbors]]
    else:
        # Notice that we look at y_train here. The neighbors will be in reference
        # to what data point were provided, which in this case is X_train/y_train
        neighboring_classes = y_train[neighbors]
        y_pred = [get_class(dists[i], neighboring_classes[i]) for i in range(len(neighboring_classes))]
    return (y_pred == y_test).mean()

In [ ]:
from sklearn.neighbors import KNeighborsClassifier, NearestNeighbors
import time

neighbors = [3, 5, 15, 101]
time_taken = {
    'maximum_class': [],
    'maximum_class_min_distance': []
}
accuracies = {
    'maximum_class': [],
    'maximum_class_min_distance': []
}

def train_knns():
    for n in neighbors:
        for key in time_taken.keys():
            print(f'training with {n} neighbors and {key} classification')
            knn = NearestNeighbors(n_neighbors=n)
            knn.fit(X_train)
            start = time.time()
            accuracy = get_accuracy(knn, n, key, X_test)
            end = time.time()
            print(f'time: {(end-start)/60:.3f} min\t\tAccuracy: {(100*accuracy):.2f}')

            time_taken[key].append(end - start)
            accuracies[key].append(accuracy)

train_knns()

In [ ]:
df1 = pd.DataFrame.from_dict(time_taken)
df2 = pd.DataFrame.from_dict(accuracies)


f, ax = plt.subplots(figsize=(10,10))
sns.lineplot(x=neighbors, y=df2['maximum_class'])
sns.lineplot(x=neighbors, y=df2['maximum_class_min_distance'])
ax.set_ylabel('Accuracy', fontdict={'fontsize': 20})
# Share the same X
ax2 = ax.twinx()
ax2.set_ylabel('Time taken (s)', fontdict={'fontsize': 20})
sns.lineplot(x=neighbors, y=df1['maximum_class'], ax=ax2)
sns.lineplot(x=neighbors, y=df1['maximum_class_min_distance'], ax=ax2)

ax.set_xlabel('K-neighbors', fontdict={'fontsize': 20})
ax.tick_params(axis='both', which='major', labelsize=15)
ax2.tick_params(axis='both', which='major', labelsize=15)
plt.legend(labels=['max class', 'max class + min dist'], fontsize=15, loc='lower right')

As you can see, the accuracies are pretty close, but using the custom metric for determining the label can be quite a bit slower. So is that it? Are we done? Nope! Let's look at two numbers in the dataset to gain intuition as to why we might want to reduce the dimensions.

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 10))
plt.gray()

ax1.imshow(X_train[0].reshape(28, 28))
ax2.imshow(X_train[1].reshape(28, 28))


Each pixel is it's own feature. If every image has a black pixel at (0, 0) then it's not a necessary feature to include. In fact, we're only really concerned with pixels (features) that have a high degree of variability since this will contain the most amount of information. We'll use something called principal components analysis (PCA) to automatically do this for us. PCA is a linear reduction technique that tries to project a feature space onto a smaller feature space in the direction that preserves maximum variance. That probably doesn't make much sense so I'll use a 3D ellipsoid to help explain. Below is code to plot a 3D ellipsoid, you can ignore it if you want.

In [ ]:
from sklearn.datasets import make_blobs
import numpy as np
import matplotlib.pyplot as plt

# Create a meshgrid for the space
nsamples = 100
x_radius, y_radius, z_radius = 20, 4, 6
space = np.linspace(0, 2 * np.pi, nsamples)
u, v = np.meshgrid(space, space)
# Create the x,y,z locations across the space
x = x_radius * np.cos(u) * np.cos(v)
y = y_radius * np.cos(u) * np.sin(v)
z = z_radius * np.sin(u)

# Stack X such that the columns are (x, y, z)
X = np.hstack((
    x.reshape(-1, 1),
    y.reshape(-1, 1),
    z.reshape(-1, 1) 
))

fig = plt.figure(figsize=(15,8))
ax = fig.add_subplot(111, projection='3d')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.scatter(X[:, 0], X[:, 1], X[:, 2])
fig.show()

Above is the 3D ellipsoid. It has 3 features/axes. The x-axis has a radius of 20, the y-axis a radius of 4 and the z-axis a radius of 6. Let's say that we wanted to project this 3D space to 2D such that we still have the most amount of information on this object as possible. You'd want to get rid of the y-axis and just have the x and z-axis. This way you retain the most information. Let's see this in action by performing PCA on this object down to 2 features.

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
pca.fit(X)
X_pca = pca.transform(X)

In [ ]:
f = plt.figure(figsize=(10,5))
plt.scatter(X_pca[:, 0], X_pca[:, 1])
plt.show()

Pretty cool, right? PCA was automatically able to project to a lower dimensioned space that maintained the most amount of overall variance. Now let's do PCA once more to go down to a single feature/axis. What do you think will happen? Will there be a line of length 12 or a line of length 40? Pause for a moment and then run the below cell.

In [ ]:
plt.figure(figsize=(10, 5))
pca = PCA(n_components=1)
pca.fit(X)
X_pca = pca.transform(X)
plt.scatter(X_pca[:, 0], np.zeros(X_pca.shape))
plt.show()

Is that what you expected? Since that axis had the most variance, it's the one you end up with. You can check the explained variance ratio as well to see the percentance of variance explained by each component that remains. In this case you'll have only one entry.

In [ ]:
pca.explained_variance_ratio_

Generally a rule of thumb is ~80% explained variance. Let's use this to construct a PCA for the data.

In [ ]:
pca = PCA(0.80, svd_solver='full')
pca.fit(X_train / 255.0)
X_train_pca = pca.transform(X_train / 255.0)

Let's check how many components (features) we ended up with.

In [ ]:
print('num components', pca.n_components_)
print('explained variance', pca.explained_variance_ratio_.sum())


From 784 features down to only 43! Let's see how the accuracies compare using KNN.

In [ ]:
neighbors = [3, 5, 15, 101]
X_test_pca = pca.transform(X_test / 255.0)

pca_accuracies = []
for k in neighbors:
    knn = NearestNeighbors(n_neighbors=k)
    knn.fit(X_train_pca)
    start = time.time()
    accuracy = get_accuracy(knn, k, 'maximum_class', X_test_pca)
    end = time.time()

    pca_accuracies.append(accuracy)
    print(f'{k} nearest: {accuracy*100:.3f}')


Let's see how these compared to the old accuracies of maximum class.

In [ ]:
df_acc = pd.concat((
    df2,
    pd.DataFrame(pca_accuracies, columns=['pca_acc'])), axis=1
)
df_acc = df_acc[['maximum_class', 'pca_acc']]

f, ax = plt.subplots(figsize=(10, 10))
sns.lineplot(x=neighbors, y=df_acc['maximum_class'])
sns.lineplot(x=neighbors, y=df_acc['pca_acc'])
ax.set_xlabel('K-neighbors', fontdict={'fontsize': 20})
ax.set_ylabel('Accuracy', fontdict={'fontsize': 20})
ax.legend(labels=['max class', 'pca'], fontsize=15, loc='lower right')
ax.tick_params(axis='both', which='major', labelsize=15)

As you can see, PCA actually performs better in terms of accuracy than using all the features. That's awesome! We get a smaller memory footprint, faster runtime since there are fewer features to calculate distance, and better accuracy. One of the nice things about going down to a smaller dimensionality is you can plot it and see the distribution of data. Let's start by seeing what accuracies we get with just 2 features. Also notice that we're normalizing the data before putting it into PCA above. This isn't especially necessary for this case since the features have the same range. Since PCA works by projeting to the direction with maximum variance, if one feature has a larger range, it might get incorrectly marked as more important. 

In [ ]:
pca = PCA(n_components=2)
pca.fit(X_train / 255.0)
X_train_pca = pca.transform(X_train / 255.0)

neighbors = [3, 5, 15, 101]
X_test_pca = pca.transform(X_test / 255.0)

pca_accuracies = []
for k in neighbors:
    knn = NearestNeighbors(n_neighbors=k)
    knn.fit(X_train_pca)
    start = time.time()
    accuracy = get_accuracy(knn, k, 'maximum_class', X_test_pca)
    end = time.time()

    pca_accuracies.append(accuracy)
    print(f'{k} nearest: {accuracy*100:.3f}')


In this case it actually benefits to have more neighbors, where previously it didn't. You can also see the accuracy is quite low. Let's take a look at this transformed data and see what we're getting.

In [ ]:
plt.figure(figsize=(9,9))
sns.scatterplot(x=X_train_pca[:1000, 0], y=X_train_pca[:1000, 1], hue=y_train[:1000],
                palette=sns.color_palette("hls", 10), s=200)


The above is the first 1000 data points. You can see they overlap quite a bit, so KNN will probably struggle here. This makes sense if you think about it. We initially had 784 features and went down to 2... so a lot of information was lost. Let's look at the explained variance.

In [ ]:
pca.explained_variance_ratio_.sum()

Previously we had 80% variance explained, and now only ~17%. So doing continuous linear projections down to 2 dimensions lost us a lot of information. Let's try a non-linear projection called `T-SNE`. T-SNE is not a linear reduction technique and won't be able to tell us a variance ratio.

In [ ]:
from sklearn.manifold import TSNE

# Create a subsample
X_sub = X_train[:1500, :] / 255.0


X_train_tsne = TSNE(learning_rate='auto', perplexity=30,
                    init='pca').fit_transform(X_sub)


Now let's plot the results of T-SNE.

In [ ]:
plt.figure(figsize=(10,10))
sns.scatterplot(x=X_train_tsne[:, 0], y=X_train_tsne[:, 1], hue=y_train[:1500],
                palette=sns.color_palette("hls", 10), s=200)
plt.legend(markerscale=2)


This actually looks like it's formed some distinguishable clusters! There's still some overlap, but it's looking a lot better. One problem with t-SNE is there is only a `fit_transform()` and no `transform()`. This isn't just an issue with sklearn's library. This is because t-SNE doesn't learn a mapping from high dimensional space to lower dimensional space. The way it learns is beyond the scope of this tutorial, but it's an iterative algorithm that tries to minimize some distance between the two spaces. It's directly learning with the data given and modifies the points in the low dimensional space. Because of this there isn't a way to really perform `transform()`. One way to get around this is to pass in the test data with the training data, then you can transform both at once. Let's see what this might look like on a subset of the training/test data with KNN.

In [ ]:
ntrain = 10000
X_train_sub = X_train[:ntrain, :] / 255.0
X_test_sub = X_test[:1500, :] / 255.0
# Combine into one
X_sub = np.row_stack((X_train_sub, X_test_sub))

# $MODIFY 1
tsne = TSNE(n_components=2, learning_rate='auto', perplexity=30,
                    init='pca').fit_transform(X_sub)

Now we can run KNN.

In [ ]:
neighbors = [3, 5, 15, 101]

for k in neighbors:
    knn = NearestNeighbors(n_neighbors=k)
    knn.fit(tsne[:ntrain])  # Add just the training data
    # Get accuracy on the test data
    neighbors = knn.kneighbors(tsne[ntrain:], n_neighbors=k, return_distance=False)
    y_pred = [np.bincount(row).argmax() for row in y_train[neighbors]]
    accuracy = (y_pred == y_test[:1500]).mean()
    print(f'KNN {k} with tsne - accuracy: {(accuracy*100):.3f}%')

### Tasks

1. Change the `n_components` of t-SNE. How do you find it effects your accuracy?

- What are some pros and cons of KNN?
- What are some pros and cons of t-SNE?

### Conclusion

In this tutorial you saw how KNN works and how to use it. You also learned how to use KNN as a classifier by simply looking at the training labels. You then learned about PCA and how to apply it to your data. You saw that you can get good results when you have a good explained variance ratio, but linearly projecting down too many features results in poor accuracy. Finally you learned about t-SNE and how you can use it to do a non-linear projection down to even just two features, which can be quite helpful for visualizing your data.

That's all for this one, see you next time!